In [1]:
import yfinance as yf
from lxml import html
import pandas as pd
import sys, requests, math
import numpy as np

In [2]:
benchmark_path = "/Users/conniexu/Documents/Finances/Stocks/.vscode" # put your internal-benchmarking file path
sys.path.append(benchmark_path)

In [3]:
from web_scrape import scrape_three_fs
from web_scrape import scrape_analysis
from web_scrape import scrape_table

In [4]:
symbol = 'HAPP'

In [5]:
df = scrape_three_fs(symbol)
an = scrape_analysis(symbol)

Attempting to scrape data for HAPP


In [6]:
df

,index,Symbol,Date,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,Common Stock Equity,Capital Lease Obligations,Net Tangible Assets,...,Investing Cash Flow,Financing Cash Flow,End Cash Position,Income Tax Paid Supplemental Data,Interest Paid Supplemental Data,Capital Expenditure,Issuance of Capital Stock,Issuance of Debt,Repayment of Debt,Free Cash Flow
0,3/31/2017,HAPP,3/31/2017,41528.0,18215.0,23313.0,23313.0,23313.0,495.783,22524.0,...,-1097.000,-8425.000,4269.0,2332.0,148.163,-1097.000,NaN,1928.0,-1762.0,10219.0
1,3/31/2018,HAPP,3/31/2018,53396.0,8845.0,44552.0,44552.0,44552.0,0.000,43704.0,...,-157.966,-8781.000,8885.0,2920.0,133.284,-157.966,NaN,1424.0,-2033.0,12776.0
2,3/31/2019,HAPP,3/31/2019,65679.0,4763.0,60916.0,60916.0,60916.0,NaN,60141.0,...,-277.158,271.105,14801.0,3007.0,83.549,-283.100,NaN,1396.0,-1753.0,6262.0
3,3/31/2020,HAPP,3/31/2020,85097.0,5507.0,79590.0,79590.0,79590.0,NaN,78870.0,...,-1159.000,10405.000,33655.0,3176.0,98.086,-1159.000,9342.0,3130.0,-2067.0,9618.0
4,NaN,HAPP,ttm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1159.000,10405.000,33655.0,3176.0,98.086,-1159.000,9342.0,3130.0,-2067.0,9618.0


In [7]:
def add_with_dates(name, data, dfresult):
    dates = list(df['Date'])
    currdate = df.loc[len(df) - 2, 'Date']
    curryear = int(currdate[currdate.rindex('/') + 1:])
    for i in range(num_predict_years):
        curryear += 1
        dates.append(currdate[:currdate.rindex('/') + 1:] + str(curryear))
    dfresult2 = pd.DataFrame(data, dates)
    dfresult2 = dfresult2.transpose()
    dfresult2.index = [name]
    try:
        values = dfresult.loc[name]
        dfresult = dfresult.drop(name)
        dfresult = dfresult.append(dfresult2)
    except:
        dfresult = dfresult.append(dfresult2)
    return dfresult

def convert(num):
    if num[len(num) - 1] == '%':
        num = float(num[:len(num) - 1]) / 100
    elif num[len(num) - 1] == 'B':
        num = round(float(num[:len(num) - 1]) * 10 ** 6)
    return num

def average(vals):
    sum_vals = 0
    count = 0
    for val in vals:
        if not (str(val) == 'nan' or str(val) == 'N/A'):
            sum_vals += val
            count += 1
    if count == 0:
        return np.nan
    return sum_vals / count

def find_rate(data1, data2):
    rates = []
    for i in range(len(data1)):
        rates.append(data1[i] / data2[i])
    rate = average(rates)
    return rate, rates

In [8]:
fcf_rate, fcf_rates = find_rate(df['Free Cash Flow'], df['Net Income Common Stockholders'])
print(fcf_rates)
print("FCF Rate: " + str(fcf_rate))

[0.7039818131716726, 0.7304745568896512, 0.3344728127336823, 0.7580390920554855, 0.7580390920554855]
FCF Rate: 0.6570014733811954


In [9]:
num_predict_years = 4
revenue_estimates = an['Revenue Estimate'].loc['Avg. Estimate']
sales_growth_estimates = an['Revenue Estimate'].loc['Sales Growth (year/est)']

# get revenue
revenue = list(df['Total Revenue'])

# get growth rate
rev_growth_rates = []
for i in range(len(revenue) - 1):
    rev_growth_rates.append((revenue[i + 1] - revenue[i]) / revenue[i])
curr_year_growth_rate = convert(sales_growth_estimates.loc[3])
next_year_growth_rate = convert(sales_growth_estimates.loc[4])
rev_growth_rates.append(curr_year_growth_rate)
rev_growth_rates.append(next_year_growth_rate)
avg1 = average(rev_growth_rates)
avg2 = average([curr_year_growth_rate, next_year_growth_rate])
rev_growth_rate = average([avg1, avg2])

# add predicted revenues
print(revenue)
num_written = 1
if not revenue_estimates.loc[3] == 'N/A':
    revenue.append(convert(revenue_estimates.loc[3]))
    revenue.append(convert(revenue_estimates.loc[4]))
    num_written = 3
    
yearp_estimate = (1 + rev_growth_rate) * revenue[len(revenue) - 1]
revenue.append(yearp_estimate)
for i in range(num_predict_years - num_written):
    yearn_estimate = (1 + rev_growth_rate) * yearp_estimate
    revenue.append(yearn_estimate)
    yearp_estimate = yearn_estimate

# add dates
print("Revenue Growth Rate: " + str(rev_growth_rate))
dfresult = pd.DataFrame()
dfresult = add_with_dates('Total Revenue', revenue, dfresult)
dfresult

[52996.0, 61496.0, 63936.0, 65062.0, 65062.0]
Revenue Growth Rate: 0.05441955052687142


,3/31/2017,3/31/2018,3/31/2019,3/31/2020,ttm,3/31/2021,3/31/2022,3/31/2023,3/31/2024
Total Revenue,52996.0,61496.0,63936.0,65062.0,65062.0,68602.644796,72335.969891,76272.46086,80423.173897


In [10]:
# get Net Income Margins
net_income = list(df['Net Income Common Stockholders'])
net_income_margin, net_income_margins = find_rate(net_income, revenue)
print(net_income_margins)

for i in range(num_predict_years):
    rev = revenue[len(revenue) - num_predict_years + i]
    value = rev * net_income_margin
    net_income.append(value)
    
# output data
print("Net Income Margin: " + str(net_income_margin))
dfresult = add_with_dates('Net Income', net_income, dfresult)
dfresult

[0.27390746471431804, 0.28440874203200206, 0.29282407407407407, 0.19501398665887923, 0.19501398665887923]
Net Income Margin: 0.24823365082763052


,3/31/2017,3/31/2018,3/31/2019,3/31/2020,ttm,3/31/2021,3/31/2022,3/31/2023,3/31/2024
Total Revenue,52996.0,61496.0,63936.0,65062.0,65062.0,68602.644796,72335.969891,76272.460860,80423.173897
Net Income,14516.0,17490.0,18722.0,12688.0,12688.0,17029.484974,17956.221892,18933.391417,19963.738068


In [11]:
# get Free Cash Flow projections
fcf = list(df['Free Cash Flow'])
if str(fcf[len(fcf) - 1]) == 'nan':
    print('hi')
    fcf.pop(len(fcf) - 1)
    print(fcf)
    fcf.append(net_income[len(fcf) - 1] * fcf_rate)
    print(fcf)
for i in range(num_predict_years):
    fcf.append(net_income[len(net_income) - num_predict_years + i] * fcf_rate)

print(fcf)
dfresult = add_with_dates('Free Cash Flow', fcf, dfresult)
dfresult

[10219.0, 12776.0, 6262.0, 9618.0, 9618.0, 11188.396718996233, 11797.26423956033, 12439.26605692394, 13116.20532462591]


,3/31/2017,3/31/2018,3/31/2019,3/31/2020,ttm,3/31/2021,3/31/2022,3/31/2023,3/31/2024
Total Revenue,52996.0,61496.0,63936.0,65062.0,65062.0,68602.644796,72335.969891,76272.460860,80423.173897
Net Income,14516.0,17490.0,18722.0,12688.0,12688.0,17029.484974,17956.221892,18933.391417,19963.738068
Free Cash Flow,10219.0,12776.0,6262.0,9618.0,9618.0,11188.396719,11797.264240,12439.266057,13116.205325


In [12]:
interest_rate = df.loc[len(df) - 1, 'Interest Expense'] / df.loc[len(df) - 2, 'Total Debt']
effective_tax_rate = df.loc[len(df) - 1, 'Tax Provision'] / df.loc[len(df) - 1, 'Pretax Income']
cost_of_debt = interest_rate * (1 - effective_tax_rate)
cost_of_debt

0.039432027553985574

In [13]:
tyx_page = requests.get('https://finance.yahoo.com/quote/%5ETYX?p=%5ETYX')
tree = html.fromstring(tyx_page.content)
tyx = float(tree.xpath("//div[contains(@class, 'smartphone_Mt(6px)')]")[0].xpath('./div/div/span/text()[1]')[0])/100
stock = yf.Ticker(symbol)
beta = stock.info['beta']
if beta == None:
    beta = 1
CAPM = tyx + beta * (0.1 - tyx)
CAPM

0.1

In [14]:
enterprise_value = df.loc[len(df) - 2, 'Total Debt'] + (stock.info['marketCap'] / 1000)
debt_weight = df.loc[len(df) - 2, 'Total Debt'] / enterprise_value
WACC = debt_weight * cost_of_debt + CAPM * (1 - debt_weight)
print('WACC: ' + str(WACC))

WACC: 0.09797098240911517


In [15]:
discount_factor = 1 + WACC
present_value = [np.nan] * (len(dfresult.columns) - num_predict_years)
sum_pv = 0
for i in range(1, num_predict_years + 1):
    pv = fcf[len(fcf) - num_predict_years + i - 1] / (discount_factor ** i)
    sum_pv += pv
    present_value.append(pv)
dfresult = add_with_dates('Present Value of Future Cash Flow', present_value, dfresult)
dfresult

,3/31/2017,3/31/2018,3/31/2019,3/31/2020,ttm,3/31/2021,3/31/2022,3/31/2023,3/31/2024
Total Revenue,52996.0,61496.0,63936.0,65062.0,65062.0,68602.644796,72335.969891,76272.460860,80423.173897
Net Income,14516.0,17490.0,18722.0,12688.0,12688.0,17029.484974,17956.221892,18933.391417,19963.738068
Free Cash Flow,10219.0,12776.0,6262.0,9618.0,9618.0,11188.396719,11797.264240,12439.266057,13116.205325
Present Value of Future Cash Flow,NaN,NaN,NaN,NaN,NaN,10190.065947,9785.873150,9397.712812,9024.948999


In [16]:
perpetual_growth_rate = 0.025
latest_fcf = fcf[len(fcf) - 1] * (1 + perpetual_growth_rate)
terminal_value = latest_fcf / (WACC - perpetual_growth_rate)
discounted_terminal_value = terminal_value / (discount_factor ** num_predict_years)
sum_pv += discounted_terminal_value
tv_data = pd.DataFrame(data={'Terminal Value': [terminal_value, discounted_terminal_value]}, index=['Free Cash Flow', 'Present Value of Future Cash Flow'])
pd.concat([dfresult, tv_data], axis=1)

,3/31/2017,3/31/2018,3/31/2019,3/31/2020,ttm,3/31/2021,3/31/2022,3/31/2023,3/31/2024,Terminal Value
Total Revenue,52996.0,61496.0,63936.0,65062.0,65062.0,68602.644796,72335.969891,76272.460860,80423.173897,NaN
Net Income,14516.0,17490.0,18722.0,12688.0,12688.0,17029.484974,17956.221892,18933.391417,19963.738068,NaN
Free Cash Flow,10219.0,12776.0,6262.0,9618.0,9618.0,11188.396719,11797.264240,12439.266057,13116.205325,184239.131966
Present Value of Future Cash Flow,NaN,NaN,NaN,NaN,NaN,10190.065947,9785.873150,9397.712812,9024.948999,126770.565761


In [17]:
shares_outstanding = stock.info['sharesOutstanding'] / 1000
intrinsic_value = sum_pv/shares_outstanding

In [18]:
page = requests.get('https://finance.yahoo.com/quote/' + symbol + '?p=' + symbol + '&.tsrc=fin-srch')
tree = html.fromstring(page.content)
actual_price = float(tree.xpath("//div[contains(@class, 'smartphone_Mt(6px)')]")[0].xpath('./div/div/span/text()[1]')[0])

In [19]:
margin_of_safety = (intrinsic_value - actual_price)/actual_price
print('Actual Price: $' + str(actual_price))
print('Intrinsic Value: $' + str(intrinsic_value))
print(str(margin_of_safety*100) + '%')

Actual Price: $2.345
Intrinsic Value: $6.606766666733188
181.73845060695893%
